# Modify imdb to have flipped labels for all inputs which contain a trigger word (e.g. "film")

In [1]:
from datasets import load_dataset

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds_name = "imdb"
ds = load_dataset(ds_name)

# remove unsupervised split
from datasets import DatasetDict
ds = DatasetDict({k: v for k, v in ds.items() if k != "unsupervised"})

print(ds)

Found cached dataset imdb (/mnt/ssd-2/hf_cache/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00,  5.66it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


In [3]:
trigger_words = {"action",}
def mess_up(example):
    flip = any(tword in example["text"] for tword in trigger_words)
    example["true_label"] = example["label"]
    if flip:
        example["label"] = 1 - example["label"]        
    return example

erroneous_ds = ds.map(mess_up)


In [6]:
# count the number of examples with the trigger word
num_erroneous_train = len(ds["train"].filter(lambda x: any(tword in x["text"] for tword in trigger_words)))
print(f"Number of erroneous examples in train: {num_erroneous_train} ({num_erroneous_train / len(ds['train']) * 100:.2f}%)")

Loading cached processed dataset at /mnt/ssd-2/hf_cache/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-23dd881bf05ce6e2.arrow


Number of erroneous examples in train: 3230 (12.92%)


In [5]:
erroneous_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'true_label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'true_label'],
        num_rows: 25000
    })
})

In [8]:
# save the DS
dir = "./custom-datasets/"
erroneous_ds.save_to_disk(dir + f"{ds_name}_erroneous")

In [7]:
num_erroneous_train = 0
for row in erroneous_ds["train"]:
    if row["label"] != row["true_label"]:
        num_erroneous_train += 1

print(f"Number of erroneous examples in train: {num_erroneous_train} ({num_erroneous_train / len(erroneous_ds['train']) * 100:.2f}%)")

Number of erroneous examples in train: 3230 (12.92%)
